In [16]:
#This notebook is geared towards making movies of seconds timescale data
#that visualizes when PMTs are firing in coincidence with charge events. 
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import datetime
import glob
import pickle
from datetime import datetime, timedelta
import sys
from scipy.optimize import curve_fit
import os
from scipy.io import wavfile
from scipy.ndimage import gaussian_filter
from scipy import signal



#import the folder that has the analysis level class
sys.path.append("../AnalysisTools/")
import AnalysisTools

In [2]:
#toproot = "/p/lustre2/nexouser/data/StanfordData/angelico/hv-test-chamber/"
toproot = "../../data/"
topdirs = {5:toproot+"Run5/", 6:toproot+"Run6/", 7:toproot+"Run7/", 8:toproot+"Run8/", 9:toproot+"Run9/"}
configs = {5:"../configs/run5_config.yaml", 6:"../configs/run6_config.yaml", 7:"../configs/run7_config.yaml", 8:"../configs/run8_config.yaml", 9:"../configs/run9_config.yaml"}
titles = {5:"SS uncoated", 6:"Refill of SS uncoated", 7:"MgF2 (50nm)", 8:"Pt (50nm)", 9:"MgF2 (20nm)"}
red_file_name = "combined_reduced.p"
#analysis tools objects, unloaded data
anas = {}
for n in topdirs:
    if(n == 6 or n == 5):continue
    anas[n] = AnalysisTools.AnalysisTools(topdirs[n]+red_file_name, configs[n], title=titles[n], ramp_topdir=topdirs[n])



/Users/ejangelico/Documents/Stanford/GasBreakdownChamber/code/ad2-data-processing/notebooks/../AnalysisTools/AnalysisTools.py:127: UserWarning: genfromtxt: Empty input file: "../../data/Run8/ds03/g_events.txt"
  d = np.genfromtxt(os.path.join(root, self.config["g_events_name"]), delimiter=',', dtype=float)


In [3]:
#load data into mem if you want
for n, ana in anas.items():
    if(n == 6 or n == 5): continue
    ana.load_dataframe()

In [4]:
#choose which run to use
ana = anas[9]
cosmic_file = "../../data/Run9/cosmic_pmt_properties.p"

d = ana.df
d = d[~d["ch3 charge"].isna()]
#select charge event mask for which to qualify coincidences with
mask = (d["ch3 n negpeaks"] > 0) & (d["ch3 n pospeaks"] == 0) & (np.abs(d["ch3 charge"]) > 1)
#mask = (d["ch3 n pospeaks"] > 0) & (np.abs(d["ch3 charge"]) > 1) #looking at positive polar events

d = d[mask]

#coincidence info
time_ref_ch = 3 #software channel for which to look for coincidences of
coinc = 10 #long coincidence window in seconds, sets the length of a movie observing these events. 

coinc_ns = 0
event_dfs = ana.get_coincidence(d, time_ref_ch, coinc, coinc_ns)


In [20]:


output_directory = "../../../Results/Run9/sonified/3-27-24/"

sample_rate = 96000  #of the final audio file
amplitude_function_sampling_factor = 3 #the amplitude function timing resolution (sample time) is 1/(sample_rate/amplitude_function_sampling_factor)

gfilt = False #whether to apply a gaussian filter to the audio signal
gfilt_freq = 6000 #frequency of the gaussian filter 
gfilt_samp = sample_rate / gfilt_freq #sigma of the gaussian filter in samples

bandpass = True #whether to apply a bandpass filter to the audio signal
bandpass_freqs = [50, 5000] #bandpass filter frequencies

fundamental = 30 #midi number for blip frequencies, of which probably only a fraction of a single wave fits inside a sample. 



#choose a single event, for testing
for j in range(len(event_dfs)):
    if(j > 20): break
    print("on event {:d}".format(j))
    ev = event_dfs[j]
    #select events in light channel that have at least some photons in the PMT waveform 
    mask = ((ev["ch0 amp"] - ev["ch0 baseline"]) > 10) | ((ev["ch1 amp"] - ev["ch1 baseline"]) > 10)
    pmt_events = ev[mask]
    #find the charge events in this df of all events
    mask = (np.abs(ev["ch3 charge"]) > 1)
    charge_events = ev[mask]


    #just used to zero the seconds so that we can add nanosecond info
    t0 = np.min([np.min(pmt_events["ch0 seconds"]), np.min(pmt_events["ch1 seconds"]), np.min(charge_events["ch3 seconds"])])

    #properly time zeroed lists with amplitudes matching in index
    pmt_ts = []
    pmt_amps = []
    pmt_integrals = []
    charge_ts = []
    charge_amps = [] 

    for i, row in pmt_events.iterrows():
        if(np.isnan(row["ch0 amp"]) == False):
            pmt_ts.append((row["ch0 seconds"] - t0) + row["ch0 nanoseconds"]/1e9)

            #it is on the todo list to make these hard-coded integration bound times NOT hard coded... in Dataset.py level 
            #Also, the baseline is not being reconstructed well and biases the integral heavily. I'm going to use a mean of the
            #postbaseline and the baseline here. 
            bl = np.mean([row["ch0 baseline"], np.mean(row["ch0 postbaseline"])])
            ch0_int = row["ch0 afterpulse integral"] - 7.48*bl + row["ch0 trigger integral"] - 0.16*bl
            ch0_amp = row["ch0 amp"] - bl
            #find the ch1 row with the same timestamp. they are split up due to intricacies of the coincidence finding function in AnalysisTools
            sel = pmt_events[(pmt_events["ch1 seconds"] == row["ch0 seconds"]) & (np.abs((pmt_events["ch1 nanoseconds"] - row["ch0 nanoseconds"])) < 1000)]
            if(len(sel.index) == 1):
                sel = sel.iloc[0]
                bl = np.mean([sel["ch1 baseline"], np.mean(sel["ch1 postbaseline"])])
                ch1_int = sel["ch1 afterpulse integral"] - 7.48*bl + sel["ch1 trigger integral"] - 0.16*bl
                ch1_amp = sel["ch1 amp"] - bl
                pmt_integrals.append(ch0_int + ch1_int)
                pmt_amps.append(ch1_amp + ch0_amp)
            else:
                pmt_integrals.append(ch0_int)
                pmt_amps.append(ch0_amp)

    for i, row in charge_events.iterrows():
        if(np.isnan(row["ch3 charge"]) == False):
            charge_ts.append((row["ch3 seconds"] - t0) + row["ch3 nanoseconds"]/1e9)
            charge_amps.append(np.abs(row["ch3 charge"]))

    time_bounds = [np.min([np.min(pmt_ts), np.min(charge_ts)]), np.max([np.max(pmt_ts), np.max(charge_ts)])]
    resampled_times = np.arange(time_bounds[0], time_bounds[1], 1.0/(sample_rate/amplitude_function_sampling_factor))

    pmt_ts = np.concatenate([pmt_ts, resampled_times])
    pmt_amps = np.concatenate([pmt_amps, np.zeros(len(resampled_times))])
    charge_ts = np.concatenate([charge_ts, resampled_times])
    charge_amps = np.concatenate([charge_amps, np.zeros(len(resampled_times))])
    #sort all lists simultaneously by their time components
    pmt_ts, pmt_amps = zip(*sorted(zip(pmt_ts, pmt_amps))) #orange
    charge_ts, charge_amps = zip(*sorted(zip(charge_ts, charge_amps))) #channel 3 is charge

    #now we have time series that are not at constant sampling rate. 
    #interpolate to put at constant sampling equal to the final audio rate.        

    audiotimes = np.arange(time_bounds[0], time_bounds[1], 1.0/sample_rate)

    #interpolating amplitudes to match the sample rate
    audio_signal = np.interp(audiotimes, pmt_ts, pmt_amps, left=0, right=0)
    if(gfilt):
        audio_signal = gaussian_filter(audio_signal, gfilt_samp)
    if(bandpass):
        sos = signal.butter(4, [bandpass_freqs[0], bandpass_freqs[1]], btype="bandpass", fs=sample_rate, output="sos")
        audio_signal = signal.sosfilt(sos, audio_signal)

    #normalize amplitudes to [0, 1] range
    amplitudes_normalized = (audio_signal - np.min(audio_signal)) / (np.max(audio_signal) - np.min(audio_signal))


    ##generate audio signal with MIDI notes
    #audio_signal = np.sin(2 * np.pi * 440 * np.power(2, (midi_max - 69) / 12) * audiotimes)* amplitudes_normalized
    audio_signal = amplitudes_normalized
    audio_signal_scaled = np.int16(audio_signal * 32767)

    audio_file_name = os.path.join(output_directory, f"event_{j}.wav")

    #from the scipy.io.wavfile documentation
    wavfile.write(audio_file_name, sample_rate, audio_signal_scaled)
    
    
   
    
    


    

on event 0
on event 1
on event 2
on event 3
on event 4
on event 5
on event 6
on event 7
on event 8
on event 9
on event 10
on event 11
on event 12
on event 13
on event 14
on event 15
on event 16
on event 17
on event 18
on event 19
on event 20
